In [1]:
#import csv file
import pandas as pd
order_brushing_df = pd.read_csv("/order_brush_order.csv")

In [2]:
order_brushing_df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [3]:
#knowing unique shopid
print(len(order_brushing_df['shopid'].unique()))

18770


In [4]:
order_groupby_shop = order_brushing_df.groupby('shopid') 

In [5]:
#changing the event_time data type into datetime
from datetime import datetime
order_brushing_df['event_time'] = order_brushing_df['event_time'].apply(lambda x:datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

In [6]:
order_brushing_df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [7]:
# #filtering more than 1 time order
# df1 = order_brushing_df.groupby('shopid').filter(lambda x: len(x) > 1)

# #filter shopid based on orderid
# df1 = df1[~df1['shopid'].isin(shop_order_statistic['shopid'])]
df1 = order_brushing_df

In [8]:
df1.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [9]:
#group by event time and shop id, and counting the amount of trasaction (based on order / user)
df1_shopid = df1.groupby([pd.Grouper(key='event_time',freq='H'),df1.shopid]).size().reset_index(name='count_order')
df1_userid = df1.groupby([pd.Grouper(key='event_time',freq='H'),df1.userid]).size().reset_index(name='count_user')

In [10]:
#group by event_time and shopid, then ccount user that do the transaction in that time
df2 = df1.groupby([pd.Grouper(key='event_time',freq='H'),df1.shopid,df1.userid]).size().reset_index(name='count')

In [11]:
df2_countuser=df2.groupby(['event_time','shopid'])['userid'].nunique().reset_index(name='count_user')

In [12]:
#group by event_time and shop id, so can get the count_order and count_user that do the transaction
df_join = pd.merge(df1_shopid,df2_countuser,on=['event_time','shopid'],how='left')
df_join

,event_time,shopid,count_order,count_user
0,2019-12-27 00:00:00,10084,2,2
1,2019-12-27 00:00:00,10151,2,2
2,2019-12-27 00:00:00,10159,1,1
3,2019-12-27 00:00:00,10208,2,2
4,2019-12-27 00:00:00,10287,3,3
...,...,...,...,...
146762,2019-12-31 23:00:00,212058195,1,1
146763,2019-12-31 23:00:00,213137628,1,1
146764,2019-12-31 23:00:00,213236358,1,1
146765,2019-12-31 23:00:00,213507904,1,1


In [13]:
#getting value of concentrate_rate
df_join['concentrate_rate'] = df_join['count_order'] / df_join['count_user']
df_join

,event_time,shopid,count_order,count_user,concentrate_rate
0,2019-12-27 00:00:00,10084,2,2,1.0
1,2019-12-27 00:00:00,10151,2,2,1.0
2,2019-12-27 00:00:00,10159,1,1,1.0
3,2019-12-27 00:00:00,10208,2,2,1.0
4,2019-12-27 00:00:00,10287,3,3,1.0
...,...,...,...,...,...
146762,2019-12-31 23:00:00,212058195,1,1,1.0
146763,2019-12-31 23:00:00,213137628,1,1,1.0
146764,2019-12-31 23:00:00,213236358,1,1,1.0
146765,2019-12-31 23:00:00,213507904,1,1,1.0


In [14]:
# df_final = pd.merge(df_join,df2,on=['event_time','shopid'],how='left')

In [15]:
#filter to get the potential of order brushing
df_brush_more_three = df_join[df_join['concentrate_rate'] >= 3]

In [16]:
df_final = pd.merge(df_brush_more_three,df2,on=['event_time','shopid'],how='left')

In [17]:
df_final = df_final[['shopid','userid']]
df_final = df_final.sort_values(by='shopid', ascending=False).reset_index()[['shopid', 'userid']]
#df_final = df_final.unique()
df_final['userid']= df_final['userid'].astype(str)

In [18]:
df_final

,shopid,userid
0,213141071,213646699
1,210197928,52867898
2,208696908,214111334
3,204225676,198662175
4,203531250,114282846
...,...,...
208,42818,170385453
209,42472,740844
210,10536,672345
211,10536,672345


In [19]:
#configure the dataframe based on the requirement of the competition
df_final_fix = df_final.groupby(['shopid'])['userid'].apply(lambda x: '&'.join(x.unique())).reset_index()

In [20]:
df_final_fix[df_final_fix['userid'].str.contains("&")]

,shopid,userid
18,8996761,162508227&13135622&2136861&215382704
29,16001939,205729485&1024838
68,64394533,194833170&92111793
98,98481320,124597967&96474917
131,136564914,178491887&191211430
138,145777302,201343856&107406&101582282
169,181009364,101832161&214208720


In [21]:
df_brush_less_three = df_join[(df_join['concentrate_rate'] < 3)]
df_brush_less_three = df_brush_less_three[~df_brush_less_three.shopid.isin(df_final_fix.shopid)]            

In [22]:
#filter only id with concentrate rate less than 3 
shopidless = df_brush_less_three['shopid'].unique()
zero = [str(0) for i in range(len(shopidless))]
df_con_less_three = pd.DataFrame(list(zip(shopidless, zero)), columns =['shopid', 'userid']) 

In [23]:
df_final_done = pd.concat([df_final_fix,df_con_less_three])

In [24]:
len(df_final_fix) + len(df_con_less_three) == len(order_brushing_df['shopid'].unique())

True

In [25]:
df_final_done = df_final_done[df_final_done['userid'].str.contains("&")].reset_index()
df_final_done = df_final_done[['shopid','userid']]

In [26]:
df_final_done

,shopid,userid
0,8996761,162508227&13135622&2136861&215382704
1,16001939,205729485&1024838
2,64394533,194833170&92111793
3,98481320,124597967&96474917
4,136564914,178491887&191211430
5,145777302,201343856&107406&101582282
6,181009364,101832161&214208720


In [27]:
df_final_done.to_csv(r'/order_brushing_sixthfloor.csv')